<a href="https://colab.research.google.com/github/Nawel-Bellil/AI---Deep-Learning--/blob/main/generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()


Saving kaggle (2) (1).json to kaggle (2) (1).json


{'kaggle (2) (1).json': b'{"username":"bellilnawel","key":"43eafbc8e7c816d7a0250c19154bab21"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d anokas/kuzushiji


Dataset URL: https://www.kaggle.com/datasets/anokas/kuzushiji
License(s): CC-BY-SA-4.0


In [ ]:
!unzip kuzushiji.zip -d ./data

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: ./data/kkanji/kkanji2/U+983C/98477d30790fdcbc.png  
  inflating: ./data/kkanji/kkanji2/U+983C/99c4713c3a0b9c68.png  
  inflating: ./data/kkanji/kkanji2/U+983C/9c0eec36af024aab.png  
  inflating: ./data/kkanji/kkanji2/U+983C/a4c8b7dd83e60028.png  
  inflating: ./data/kkanji/kkanji2/U+983C/a86c00411119f95f.png  
  inflating: ./data/kkanji/kkanji2/U+983C/b84341d95a612693.png  
  inflating: ./data/kkanji/kkanji2/U+983C/bad9800f26a0a1c4.png  
  inflating: ./data/kkanji/kkanji2/U+983C/c58c9522e6569cc1.png  
  inflating: ./data/kkanji/kkanji2/U+983C/c66a87daf6423151.png  
  inflating: ./data/kkanji/kkanji2/U+983C/c67d6edf7d8d5863.png  
  inflating: ./data/kkanji/kkanji2/U+983C/c7edb0636ab5832a.png  
  inflating: ./data/kkanji/kkanji2/U+983C/ceef2dbc7c304a64.png  
  inflating: ./data/kkanji/kkanji2/U+983C/d5af1dcc066eab72.png  
  inflating: ./data/kkanji/kkanji2/U+983C/daf545c68a994148.png  
  inflating:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
filepath = './data/'

x_train = np.load(filepath + 'k49-train-imgs.npz')['arr_0'].astype('float32') / 255.
x_test  = np.load(filepath + 'k49-test-imgs.npz')['arr_0'].astype('float32') / 255.


In [ ]:

# Flatten images into vectors of size 784
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test  = np.reshape(x_test, (-1, 28, 28, 1))

In [ ]:
encoding_dim = 32  # Latent space size

input_img = Input(shape=(28, 28, 1))
flat = Flatten()(input_img)
encoded = Dense(256, activation='relu')(flat)  # Increased size to 256
encoded = Dense(128, activation='relu')(encoded)  # Increased size to 128
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l2(1e-5))(encoded)  # L2 Regularization

In [ ]:

decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)  # Increased size to 256
decoded = Dense(784, activation='sigmoid')(decoded)
decoded = Reshape((28, 28, 1))(decoded)

autoencoder = Model(input_img, decoded)

In [ ]:
# Using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
datagen.fit(x_train)

autoencoder.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')  # Changed loss function

autoencoder.fit(datagen.flow(x_train, x_train, batch_size=128),  # Adjusted batch size
                epochs=50,
                validation_data=(x_test, x_test),
                callbacks=[early_stopping])

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 271/1816 ━━━━━━━━━━━━━━━━━━━━ 1:34 61ms/step - loss: 0.1822

KeyboardInterrupt: 

In [ ]:
decoded_imgs = autoencoder.predict(x_test[:10])

plt.figure(figsize=(20, 4))
for i in range(10):
    # Original
    ax = plt.subplot(2, 10, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

    # Reconstructed
    ax = plt.subplot(2, 10, i + 11)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)  # Increased filters to 64
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.3)(x)  # Increased dropout rate to 0.3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)  # Increased filters to 128
x = BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)


In [ ]:
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)


In [ ]:
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_autoencoder.h5', monitor='val_loss', save_best_only=True, mode='min')
]


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
datagen.fit(x_train)

autoencoder.fit(datagen.flow(x_train, x_train, batch_size=64),
                epochs=100,  # Increased number of epochs to 100
                validation_data=(x_test, x_test),
                callbacks=callbacks)

Epoch 1/100
 319/3631 ━━━━━━━━━━━━━━━━━━━━ 32:48 594ms/step - loss: 0.1692

In [ ]:
decoded_imgs = autoencoder.predict(x_test[:10])

plt.figure(figsize=(20, 4))
for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.axis('off')

    ax = plt.subplot(2, 10, i + 11)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.show()